# Clustering analysis of PubTrends output

Analyze different versions of clustering


In [ ]:
import gzip
import json
import logging
import os

from bokeh.plotting import output_notebook

from pysrc.config import PubtrendsConfig
from pysrc.papers.utils import SORT_MOST_CITED

SEARCH_SORT = SORT_MOST_CITED
SEARCH_PAPERS = 10_000

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s: %(message)s')
logger = logging.getLogger('notebook')

output_notebook()

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Load data

In [ ]:
from pysrc.papers.data import AnalysisData

path_data = os.path.expanduser('~/Downloads/pubmed-human-aging.json.gz')
# path_data = os.path.expanduser('~/Downloads/pubmed-human-aging-semantic.json.gz')
with gzip.open(path_data, 'r') as f:
    data = AnalysisData.from_json(json.loads(f.read().decode('utf-8')))
data

# Analysis - Phenograph algorithm (default)

In [ ]:
from pysrc.papers.analysis.graph import build_papers_graph, similarity

config = PubtrendsConfig(test=False)

bibliographic_graph = build_papers_graph(
    data.df, data.cit_df, data.cocit_grouped_df, data.bibliographic_coupling_df,
)
logger.debug(f'Bibliographic edges/nodes={bibliographic_graph.number_of_edges() / bibliographic_graph.number_of_nodes()}')

In [ ]:
from pysrc.papers.analysis.text import chunks_to_text_embeddings

papers_text_embeddings = chunks_to_text_embeddings(data.df, data.chunks_embeddings, data.chunks_idx)

In [ ]:
from pysrc.papers.analysis.node2vec import node2vec
from pysrc.config import NODE2VEC_GRAPH_EDGES, GRAPH_TEXT_SIMILARITY_EDGES
from pysrc.papers.analysis.graph import sparse_graph, add_text_similarities_edges


logger.debug('Adding text similarities edges')
add_text_similarities_edges(data.df['id'], papers_text_embeddings, bibliographic_graph, GRAPH_TEXT_SIMILARITY_EDGES)
logger.debug(f'Bibliographic+text edges/nodes={bibliographic_graph.number_of_edges() / bibliographic_graph.number_of_nodes()}')

logger.debug('Prepare sparse graph of text + bibliographic edges for node2vec')
for i, j in bibliographic_graph.edges():
    bibliographic_graph[i][j]['similarity'] = similarity(bibliographic_graph.get_edge_data(i, j))
analysis_graph = sparse_graph(bibliographic_graph, NODE2VEC_GRAPH_EDGES)


In [ ]:
graph_embeddings = node2vec(
    data.df['id'],
    analysis_graph,
    key='similarity'
)

In [ ]:
from pysrc.papers.analysis.clustering import cluster_and_sort

data.clusters, data.dendrogram = cluster_and_sort(graph_embeddings, 10)
data.df['comp'] = data.clusters

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from pysrc.config import PCA_VARIANCE

def update_coords(df, embeddings):
    pca = PCA(n_components=PCA_VARIANCE, svd_solver="full")
    pca_coords = pca.fit_transform(StandardScaler().fit_transform(embeddings))
    logger.debug('Apply transformation')
    tsne = TSNE(n_components=2, random_state=42, metric="cosine", perplexity=min(30, len(data.df) - 1))
    coords = tsne.fit_transform(pca_coords)
    df['x'] = coords[:, 0]
    df['y'] = coords[:, 1]

In [ ]:
update_coords(data.df, graph_embeddings)

In [ ]:
from pysrc.papers.plot.plotter import Plotter
from bokeh.io import show

plotter = Plotter(config, data)

In [ ]:
show(plotter.plot_papers_graph())

# Analysis - combine text + graph embeddings

In [ ]:
from pysrc.papers.utils import l2norm
from pysrc.papers.analysis.node2vec import node2vec
from pysrc.config import NODE2VEC_GRAPH_EDGES
from pysrc.papers.analysis.graph import sparse_graph

bibliographic_graph = build_papers_graph(
    data.df, data.cit_df, data.cocit_grouped_df, data.bibliographic_coupling_df,
)
logger.debug(f'Bibliographic edges/nodes={bibliographic_graph.number_of_edges() / bibliographic_graph.number_of_nodes()}')

for i, j, d in bibliographic_graph.edges(data=True):
    d['similarity'] = similarity(d)

logger.debug('Preparing sparse graph for embeddings')
node2vec_graph = sparse_graph(bibliographic_graph, NODE2VEC_GRAPH_EDGES)

logger.debug('Analyzing papers graph embeddings')
bibliographic_graph_embeddings = l2norm(node2vec(
    data.df['id'],
    node2vec_graph,
    'similarity'
))

In [ ]:
from pysrc.config import VISUALIZATION_GRAPH_EDGES

logger.debug('Adding text similarities edges')
add_text_similarities_edges(data.df['id'], papers_text_embeddings, bibliographic_graph, GRAPH_TEXT_SIMILARITY_EDGES)
logger.debug(f'Bibliographic+text edges/nodes='
             f'{bibliographic_graph.number_of_edges() / bibliographic_graph.number_of_nodes()}')
for i, j, d in bibliographic_graph.edges(data=True):
    d['similarity'] = similarity(d)
papers_graph = sparse_graph(bibliographic_graph, VISUALIZATION_GRAPH_EDGES)

# Graph only embeddings

In [ ]:
papers_embeddings = bibliographic_graph_embeddings
data.clusters, data.dendrogram = cluster_and_sort(papers_embeddings, 10)
data.df['comp'] = data.clusters
update_coords(data.df, papers_embeddings)
plotter = Plotter(config, data)
show(plotter.plot_papers_graph())


# Clustering of text embeddings vs PCA on text embeddings

In [ ]:
papers_embeddings = papers_text_embeddings
data.clusters, data.dendrogram = cluster_and_sort(papers_embeddings, 10)
data.df['comp'] = data.clusters
update_coords(data.df, papers_embeddings)
plotter = Plotter(config, data)
show(plotter.plot_papers_graph())


In [ ]:
from pysrc.config import NODE2VEC_EMBEDDINGS_VECTOR_LENGTH
from sklearn.preprocessing import normalize


# Try PCA on text embeddings - better for clustering
pca_papers_text_embeddings = PCA(n_components=NODE2VEC_EMBEDDINGS_VECTOR_LENGTH, svd_solver="full").fit_transform(
    normalize(papers_text_embeddings)
)
papers_embeddings = pca_papers_text_embeddings
data.clusters, data.dendrogram = cluster_and_sort(papers_embeddings, 10)
data.df['comp'] = data.clusters
update_coords(data.df, papers_embeddings)
plotter = Plotter(config, data)
show(plotter.plot_papers_graph())

# Text and graph embeddings

In [ ]:
import numpy as np

logger.debug('Computing aggregated graph and text embeddings for papers')
papers_embeddings = normalize(np.concatenate([normalize(bibliographic_graph_embeddings) * 2,
                                              normalize(pca_papers_text_embeddings) * 1], axis=1))
data.clusters, data.dendrogram = cluster_and_sort(papers_embeddings, 10)
data.df['comp'] = data.clusters
update_coords(data.df, papers_embeddings)
plotter = Plotter(config, data)
show(plotter.plot_papers_graph())


# END